In [1]:
checkpoint = "/home/pzhu/data/qa/squad2_model"
predict_file = "data/squad2/dev-v2.0.json"
device = "cuda:0"

In [2]:
from pytorch_transformers import XLNetForQuestionAnswering
model = XLNetForQuestionAnswering.from_pretrained(checkpoint)
model.to(device)
model.eval()
print("loaded")

loaded


In [3]:
from xlnet_qa.squad2_reader import SQuAD2Reader

reader = SQuAD2Reader(is_training=False)
examples, features, datasets = reader.squad_data(predict_file)

In [4]:
from tqdm import tqdm
import torch
from torch.utils.data import SequentialSampler, DataLoader
from xlnet_qa.utils_squad import RawResultExtended, write_predictions_extended

sampler = SequentialSampler(datasets)
dataloader = DataLoader(datasets, sampler=sampler, batch_size=1)

def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [22]:
data = tuple(t.to(device) for t in next(iter(dataloader)))
example = examples[data[3].item()]
feature = features[data[3].item()]

In [23]:
print(example.question_text)
print(example.doc_tokens)
print(example.orig_answer_text)
print(example.start_position, example.end_position)

In what country is Normandy located?
['The', 'Normans', '(Norman:', 'Nourmands;', 'French:', 'Normands;', 'Latin:', 'Normanni)', 'were', 'the', 'people', 'who', 'in', 'the', '10th', 'and', '11th', 'centuries', 'gave', 'their', 'name', 'to', 'Normandy,', 'a', 'region', 'in', 'France.', 'They', 'were', 'descended', 'from', 'Norse', '("Norman"', 'comes', 'from', '"Norseman")', 'raiders', 'and', 'pirates', 'from', 'Denmark,', 'Iceland', 'and', 'Norway', 'who,', 'under', 'their', 'leader', 'Rollo,', 'agreed', 'to', 'swear', 'fealty', 'to', 'King', 'Charles', 'III', 'of', 'West', 'Francia.', 'Through', 'generations', 'of', 'assimilation', 'and', 'mixing', 'with', 'the', 'native', 'Frankish', 'and', 'Roman-Gaulish', 'populations,', 'their', 'descendants', 'would', 'gradually', 'merge', 'with', 'the', 'Carolingian-based', 'cultures', 'of', 'West', 'Francia.', 'The', 'distinct', 'cultural', 'and', 'ethnic', 'identity', 'of', 'the', 'Normans', 'emerged', 'initially', 'in', 'the', 'first', 'half'

In [26]:
outputs = model(input_ids = data[0],
                        attention_mask = data[1],
                        token_type_ids = data[2],
                        cls_index = data[4],
                        p_mask = data[5]
                       )

In [28]:
unique_id = int(feature.unique_id)
result = RawResultExtended(unique_id= unique_id,
                            start_top_log_probs  = to_list(outputs[0][0]),
                            start_top_index      = to_list(outputs[1][0]),
                            end_top_log_probs    = to_list(outputs[2][0]),
                            end_top_index        = to_list(outputs[3][0]),
                            cls_logits           = to_list(outputs[4][0])
                            )

In [31]:
result

RawResultExtended(unique_id=1000000000, start_top_log_probs=[0.00584795419126749, 0.00584795419126749, 0.005847953725606203, 0.005847953725606203, 0.005847953725606203], start_top_index=[164, 153, 47, 15, 11], end_top_log_probs=[0.005847962573170662, 0.005847966764122248, 0.0058479635044932365, 0.0058479649014770985, 0.005847963970154524, 0.005847962573170662, 0.0058479611761868, 0.0058479635044932365, 0.005847962107509375, 0.0058479611761868, 0.005847962573170662, 0.0058479611761868, 0.0058479635044932365, 0.005847962107509375, 0.0058479611761868, 0.005847959779202938, 0.0058479611761868, 0.005847960710525513, 0.005847962107509375, 0.0058479611761868, 0.005847959779202938, 0.0058479611761868, 0.005847960710525513, 0.005847959313541651, 0.0058479611761868], end_top_index=[56, 46, 64, 46, 46, 46, 51, 47, 52, 75, 42, 47, 46, 45, 52, 52, 52, 54, 51, 144, 47, 50, 52, 16, 69], cls_logits=-0.8711456656455994)

In [21]:
data[3].item()

0